<a href="https://colab.research.google.com/github/kutyadog/ai_notebooks/blob/main/AI_seo_POC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ai-SEO-tests

Dec. 2023. - Early stage POC - SEO Analysis driven by AI. Some functionality removed for legal reasons.

Project was demo'd & well recieved; advanced to a production level and launched July 2024.

Questions regarding code, please feel free to reach out at kutyadog@gmail.com (Chris Johnson).

NOTE: Not all functions here will work outside wapo VPN. Keys & internal URLs removed for obvious security reasons.




In [ ]:
# @title Start it up!

!pip install -q gradio==3.41.0
# !pip install -q gradio
# !pip install https://gradio-builds.s3.amazonaws.com/fbcd9a9acef733f106b5138a61fce2190c12c76c/gradio-3.45.1-py3-none-any.whl
!pip install beautifulsoup4
# !pip install rake-nltk
!pip install keybert
!pip install -U textblob
!python -m textblob.download_corpora
!pip install -q openai
!pip install pytrends

import gradio
import requests
import pandas as pd
import math

from bs4 import BeautifulSoup

import gradio
import requests
import json

from keybert import KeyBERT
from bs4 import BeautifulSoup

import openai
from google.colab import userdata
openai.api_key = userdata.get('OPENAI_API_KEY')

# globals!
search_results_array = [] #gr.Array()
open_ai_model = "gpt-4-1106-preview"
open_ai_max_tokens = 8500
open_ai_temp = 0.0

## Necessary Functions

In [ ]:
# @title Story Utility functions

def get_story_headline(element):
  try:
    item = element['headlines']['basic']
  except KeyError:
    item = ''
  return str(item)

def get_story_title(element):
  try:
    item = element['headlines']['meta_title'] # + " - The Washington Post"
  except KeyError:
    item = ''
  if item == '':
    try:
      item = element['headlines']['basic']
    except KeyError:
      item = ''
  return str(item)

def get_story_description(element):
  try:
    item = element['description']['basic']
  except KeyError:
    item = ''
  return str(item)

def get_id(element):
  try:
    item = element['_id']
  except KeyError:
    item = ''
  return str(item)

def get_content_elements(element):
  try:
    item = element['content_elements']
  except KeyError:
    item = ''
  return item

def get_story_section_path(element):
  try:
    item = element['taxonomy.primary_section.path']
  except KeyError:
    item = ''

  if item == '':
    try:
      item = element['additional_properties']['commercial_node']
    except KeyError:
      item = ''

  if item == '':
    try:
      item = element['taxonomy']['primary_site']['_id']
    except KeyError:
      item = ''

  return str(item)

def get_story_canonical_url(element):
  try:
    item = element['headlines']['url']
  except KeyError:
    item = ''

  if item == '':
    try:
      item = element['canonical_url']
    except KeyError:
      item = ''

  return str(item)

def get_story_publish_date(element):
  try:
    item = element['publish_date']
  except KeyError:
    item = ''
  return str(item)

def get_story_slug(element):
  try:
    item = element['slug']
  except KeyError:
    item = ''
  return str(item)

def get_story_type(element):
  try:
    item = element['type']
  except KeyError:
    item = ''
  return str(item)

def get_story_subtype(element):
  try:
    item = element['subtype']
  except KeyError:
    item = ''
  return str(item)

def count_keywords(this_string, keywordsArray):
  count = 0
  for keyword in keywordsArray:
    if keyword.lower() in str(this_string).lower():
      count += 1
  return count

In [ ]:
# @title Prism functions

from google.colab import userdata

prism_key = userdata.get('WAPO_PRISM_KEY')

def get_prism_data_from_url(url):
  fixed_url = url.replace('https://www.washingtonpost.com', '')
  fixed_url = fixed_url.replace('https://washingtonpost.com', '')
  if fixed_url[-1] != '/' and not fixed_url.endswith("_story.html") :
    fixed_url += '/'
  fullpath = f"https://prism.ext.nile.works/content/v4?website=washpost&key={prism_key}&website_url={fixed_url}&website=washpost"
  response = requests.get(fullpath)
  my_json_object = json.loads(response.content)
  return my_json_object

# story_object = get_prism_data_from_url("https://www.washingtonpost.com/sports/2023/10/01/taylor-swift-kelce-chiefs-jets/")



def get_prism_search_data_by_query(query):
  # CHRIS THIS IS THE NEW PATH WITH ONLY FIELDS YOU WANT:
  # https://prism.wpit.nile.works/content/v4/search/?website=washpost&sort=last_updated_date:desc&from=0&size=20&body=%7B%22query%22:%7B%22bool%22:%7B%22must%22:%5B%7B%22term%22:%7B%22type%22:%22story%22%7D%7D,%7B%22match_phrase_prefix%22:%7B%22slug%22:%22linkbox%22%7D%7D,%7B%22term%22:%7B%22revision.published%22:%22true%22%7D%7D,%7B%22range%22:%7B%22last_updated_date%22:%7B%7D%7D%7D,%7B%22match_phrase_prefix%22:%7B%22source.system%22:%22ellipsis%22%7D%7D%5D,%22must_not%22:%5B%7B%22match_phrase%22:%7B%22subtype%22:%22live-all%22%7D%7D,%7B%22match_phrase%22:%7B%22subtype%22:%22live-update%22%7D%7D,%7B%22match_phrase%22:%7B%22subtype%22:%22live-reporter-insight%22%7D%7D,%7B%22match_phrase%22:%7B%22subtype%22:%22live-race-call%22%7D%7D,%7B%22match_phrase%22:%7B%22subtype%22:%22live-faq%22%7D%7D,%7B%22match_phrase%22:%7B%22taxonomy.tags.slug%22:%22ellipsis-template%22%7D%7D%5D%7D%7D%7D&&_sourceInclude=headlines.basic,headlines.meta_title,headlines.url,canonical_url,content_elements.type,content_elements.content,additional_properties.commercial_node,description.basic,publish_date,slug,type,subtype,taxonomy.primary_section.path,taxonomy.primary_section._id,website_url

  #https://prism.ext.nile.works/content/v4/search/published?q=+headlines.basic:"lewiston"+subtype:"default"&nocache=true&s=date&from=0&t=story&size=30&sort=display_date:desc&website=washpost&key={prism_key}
  # query_text = 'q=+headlines.basic:"'+ query +'" AND (subtype:default)'
  query_text = 'q=+headlines.basic:"'+ query +'" AND !(subtype:live-all) AND !(subtype:live-update) AND !(subtype:live-reporter-insight)'
  query_text += ' AND !(canonical_url:"")'
  query_text += ' AND (type:"story")'

  fullpath = 'https://prism.ext.nile.works/content/v4/search/published?{}&nocache=true&s=date&from=0&t=story&size=30&sort=display_date:desc&website=washpost&key={}'.format(
      query_text,
      prism_key
  )

  # +subtype:"default" # removes live
  # print(fullpath)
  response = requests.get(fullpath)
  my_json_object = json.loads(response.content)
  return my_json_object

# search_object = get_prism_search_data_by_query("Taylor Swift comes to NFL")
# search_object


# get_prism_search_data_by_query('taylor swift')
# https://prism.wpit.nile.works/content/v4/search/?q=+headlines.basic:"Taylor Swift comes to"&website=washpost
# https://prism.wpit.nile.works/content/v4/search/?q=%2Bheadlines.basic%3A%22Taylor%20Swift%20comes%20to%22&website=washpost
# https://prism.ext.nile.works/content/v4/search/?q=%2Bheadlines.basic%3A%22Taylor%20Swift%20comes%20to%22&website=washpost&key={prism_key}

In [ ]:
# @title Interface functions

def get_html_score_from_url(url):
  global story_object
  story_object = get_prism_data_from_url(url)
  return get_html_score_from_story_object(story_object)

def get_html_score_from_story_object(this_story_object):
  global score_object, ai_score_object
  score_object, this_story_object = analyze_seo_by_story_object(this_story_object)
  story_html_string = get_full_story_html_api(this_story_object)

  # ----------------RECIRC LINKS -------------- ONLY call if score is yellow or Red!!!
  reCircLinks = get_recirc_links_from_storyObject(this_story_object) # if score_object['recirc_links']['score'] != "Green" else []

  # ----------------AI Story summary #2 --------------
  ai_score_object = ai_score_story(this_story_object, story_html_string)
  if "error" in ai_score_object:
      print(ai_score_object['error'])
      return(f"----ERROR: fn: ai_score_story - OpenAI API returned an API Error: {ai_score_object}")

  ai_score_object = score_seo_keywords(ai_score_object, this_story_object)

  # {
      # 'keywords': 'Hamas, Israel, Gaza, attack, Palestinian, Iran',
      # 'headline_score': 'Yellow',
      # 'headline_comment': 'The headline could be more specific about the nature of the attack and its implications.',
      # 'meta_title_score': 'Yellow',
      # 'meta_title_comment': 'The meta title should include the year of the attack for clarity and to differentiate from past events.',
      # 'meta_description_score': 'Yellow',
      # 'meta_description_comment': 'The meta description is too vague and does not provide enough detail about the recent events or the historical context.',
      # 'meta_descriptions': "Exploring the reasons behind Hamas's 2023 attack on Israel and the escalating conflict.
      #     ** A detailed look at the Gaza-Israel conflict following Hamas's recent attack.
      #     ** Unpacking the latest surge in violence between Hamas and Israel, and the global implications."
  # }

  return convert_score_object_to_html(score_object, reCircLinks, this_story_object, ai_score_object)

# -------- usage example:
# response_object = get_recirc_links_from_ai("Taylor Swift comes to Chiefs-Jets, and the NFL gets an even bigger spotlight", "/sports", "https://www.washingtonpost.com/sports/2023/10/01/taylor-swift-kelce-chiefs-jets/")
# response_object[0]





def interface_search_stories_by_query(xQuery):
  print("interface_search_stories_by_query")
  global search_results_array
  search_results = get_prism_search_data_by_query(xQuery)
  search_results_array = search_results['content_elements']
  xArray = []
  for element in search_results_array:
    headline = get_story_headline( element )
    item_type = get_story_type( element )
    item_subtype = get_story_subtype(element)
    canon_url = get_story_canonical_url(element)
    xArray.append({
        "Id": get_id(element),
        "Headline": headline,
        "Slug": get_story_slug(element),
        "Published data": get_story_publish_date(element)
    })
  df = pd.DataFrame(xArray)
  return df

# -------- usage example:
# interface_search_stories_by_query("Lewiston")



In [ ]:
# @title SEO scoring functions



story_object = ''

def analyze_seo_by_url(url):
  story_object = get_prism_data_from_url(url)
  print( 'analyze_seo_by_url')
  print(story_object)

  return analyze_seo_by_story_object(story_object)

def analyze_seo_by_story_object(story_object):
  internal_links_score = _analyze_internal_links(story_object)
  print( 'internal_links_score')
  print(internal_links_score)
  # Analyze the RECIRC links.
  recirc_links_score = _analyze_recirc_links(story_object)

  # Analyze the headline.
  headline_score = _analyze_headline(story_object)
  meta_title_score = _analyze_title(story_object)
  meta_description_score = _analyze_meta_description(story_object)

  # Return the SEO scores.
  score_object = {
      "internal_links": internal_links_score,
      "recirc_links" : recirc_links_score,
      "headline": headline_score,
      "title": meta_title_score,
      "meta_description": meta_description_score,
  }
  return score_object, story_object
  # return render_results(score_object)

def _analyze_headline(story_object):
  # Get the length of the headline.
    headline = get_story_headline( story_object )
    headline_length = len(headline)

    # Return the score for the headline.
    score = "Red"
    if headline_length <= 70:
      score = "Green"
    elif headline_length <= 80:
      score = "Yellow"

    head_suggest = '<span class="'+ score +'">&#10008;</span><strong style="margin-left:6px;">Length:</strong> 70 characters or less. ({} chars)'.format(headline_length) if (score != "Green") else '<span class="Green">&#10004;</span><strong style="margin-left:6px;">Length:</strong> 70 characters or less. ({} chars)'.format(headline_length)
    return {'score': score, 'score_value': headline_length, 'value': headline, 'suggestion': head_suggest}

# -------- usage example:
# _analyze_headline(story_object)

def _analyze_title(story_object):
  meta_title = get_story_title( story_object )
  meta_title_length = len(meta_title)
  # The ideal length for the meta title is 50–60 characters. Google starts to cut off the title tag after 50-60 characters.

  # Return the score for the headline.
  score = "Red"
  if meta_title_length <= 60 and meta_title_length >= 50:
    score = "Green"
  elif meta_title_length <= 70 and meta_title_length >= 40:
    score = "Yellow"

  meta_title_suggest = '<span class="'+ score +'">&#10008;</span><strong style="margin-left:6px;">Length:</strong> 50–60 characters. ({} chars)'.format(meta_title_length) if (score != "Green") else '<span class="Green">&#10004;</span><strong style="margin-left:6px;">Length:</strong> 50–60 characters. ({} chars)'.format(meta_title_length)
  if meta_title_length == 0:
    score = "Red"
    meta_title_suggest = '<div><span class="Red">&#10008;</span><strong style="margin-left:6px;">Meta title is missing.</strong></div>'

  return {'score': score, 'score_value': meta_title_length, 'value': meta_title, 'suggestion': meta_title_suggest}

# -------- usage example:
# _analyze_title(story_object)

def _analyze_meta_description(story_object):
  # Get the length of the meta description.
  meta_description = get_story_description( story_object )
  meta_description_length = len(meta_description)

  #Return the score for the meta description.
  score = "Red"
  if meta_description_length <= 160 and meta_description_length > 50:
    score = "Green"
  elif meta_description_length <= 180 and meta_description_length > 160:
    score = "Yellow"
  desc_suggest = '<span class="'+ score +'">&#10008;</span><strong style="margin-left:6px;">Length:</strong> 50-160 characters. ({} chars)'.format(meta_description_length) if (score != "Green") else '<span class="Green">&#10004;</span><strong style="margin-left:6px;">Length:</strong> 50-160 characters. ({} chars)'.format(meta_description_length)
  return {'score': score, 'score_value': meta_description_length, 'value': meta_description, 'suggestion': desc_suggest}

# -------- usage example:
# _analyze_meta_description(story_object)

def _analyze_internal_links(story_object):
  print(story_object)
  paragraph_count = 0
  link_urls_array = []
  for element in get_content_elements(story_object):
    if ( element['type'] == "text" ):
      paragraph_count = paragraph_count + 1
      if "<a href=" in element['content']:
        soup = BeautifulSoup(element['content'], "html.parser")
        for link in soup.find_all('a'):
          link_href = link.get('href')
          if "washingtonpost.com" in link_href:
            print(link_href)
            link_urls_array.append(link_href)

  # Count the number of internal links.
  num_internal_links = len(link_urls_array)
  print("num_internal_links: " + str(num_internal_links))
  # Calculate the average number of paragraphs per internal link.
  avg_paragraphs_per_internal_link = paragraph_count / num_internal_links if (num_internal_links > 0) else 0

  score = "Red"
  if avg_paragraphs_per_internal_link <= 5:
    score = "Green"
  elif avg_paragraphs_per_internal_link <= 10:
    score = "Yellow"

  need_to_add = math.ceil( ( paragraph_count / 5 ) - num_internal_links )
  print("need_to_add: " + str(need_to_add))
  valueObject = {'paragraph_count': paragraph_count, 'num_internal_links': num_internal_links, 'avg_paragraphs_per_internal_link': avg_paragraphs_per_internal_link, 'need_to_add': need_to_add }
  print("valueObject: ")
  print(valueObject)
  return {'score': score, 'value': valueObject, 'suggestion': 'Ideally have 1 internal link per 5 paragraphs. You currently have 1 internal link per {} paragraphs.'.format(avg_paragraphs_per_internal_link)}

def _analyze_recirc_links(story_object):
  interstitial_array = []
  carousel_array = []
  paragraph_count = 0

  for element in get_content_elements(story_object):
    # print(element['type'])
    if ( element['type'] == "custom_embed" and element['subtype'] == "magnet" and element['embed']['config']['subtype'] == "collection" ):
      # print("Found story carousel")
      # print(element['subtype'])
      # print(element['embed']['config']['subtype'])
      carousel_array.append( { 'type': 'carousel', 'data': element['embed']['url'], 'title' : element['embed']['config']['title'] })

    elif ( element['type'] == "interstitial_link" ):
      interstitial_array.append( { 'type': 'interstitial', 'data': element['url'], 'title' : element['content']})
      # print(element)
    elif ( element['type'] == "text" ):
      paragraph_count = paragraph_count + 1

  num_recircs_links = len(interstitial_array) + len(carousel_array)

  score = "Red"
  if num_recircs_links >= 2:
    score = "Green"
  elif num_recircs_links == 1:
    score = "Yellow"

  need_to_add = max( [0, 2 - ( len(interstitial_array) + len(carousel_array) )]  )
  return {'score': score, 'need_to_add': need_to_add, 'value': 'You currently have {} interstitials and {} story carousels for your {} paragraph story.'.format(len(interstitial_array), len(carousel_array), paragraph_count), 'suggestion': 'Ideally have at least 2 interstitials or story carousels for articles at least 12 paragraphs long.'}

keywords = []
def score_seo_keywords(this_ai_score_object, this_story_object):
  global keywords
  headline = get_story_headline(this_story_object),
  title = get_story_title( this_story_object ),
  description = get_story_description( this_story_object )
  canonical_url = get_story_canonical_url( this_story_object )

  keywords = ai_score_object['keywords']

  headline_keyword_cnt = count_keywords(headline, keywords)
  title_keyword_cnt = count_keywords(title, keywords)
  description_keyword_cnt = count_keywords(description, keywords)
  canonical_url_keyword_cnt = count_keywords(canonical_url, keywords)
  print("canonical_url_keyword_cnt: "+ str(canonical_url_keyword_cnt))
  def score_by_cnt(xcount):
    if xcount > 2:
      return "Green", ""
    elif xcount > 0:
      return "Yellow", "Add more SEO keywords."
    return "Red", "Add more SEO keywords."

  this_ai_score_object['headline_score'], this_ai_score_object['headline_comment'] = score_by_cnt(headline_keyword_cnt)
  this_ai_score_object['meta_title_score'], this_ai_score_object['meta_title_comment'] = score_by_cnt(title_keyword_cnt)
  this_ai_score_object['meta_description_score'], this_ai_score_object['meta_description_comment'] = score_by_cnt(description_keyword_cnt)
  this_ai_score_object['canonical_url_score'], this_ai_score_object['canonical_url_comment'] = score_by_cnt(canonical_url_keyword_cnt)


  # return headline_keyword_cnt, title_keyword_cnt, description_keyword_cnt
  # for item in ai_score_object['keywords']:
  # count_keywords(string, keywords)

  return this_ai_score_object



# story_object = get_prism_data_from_url('https://www.washingtonpost.com/opinions/2023/10/17/jim-jordan-speaker-house-chaos/')
# _analyze_recirc_links(story_object)

# --- prism link of stories in collection / story carousel
#     https://prism.wpit.nile.works/content/v4/collections?content_alias=opgaza

# ----------------------- DATA FOR CAROUSEL
# {'_id': 'MFNZXPFAWBEQNHU7VVPTHW7RVM',
#  'additional_properties': {
#     '_id': 'MMMHPOKNURAUDFMUIPC7IE3VG4'
#   },
#  'embed': {
#     'config': {
#         'cdn_url': 'https://prism.wpit.nile.works/content/v4/collections?content_alias=opgaza',
#         'feed': 'opgaza',
#         'subtype': 'collection',
#         'title': 'Opinions on the Israel-Gaza war'},
#     'id': 'opgaza',
#     'url': 'https://prism.wpit.nile.works/content/v4/collections?content_alias=opgaza'
#   },
#  'subtype': 'magnet',
#  'type': 'custom_embed'
# }



In [ ]:
# @title Interface HTML Functions



# ---- AI response:

# {
#     "keywords": [
#         "marijuana restrictions",
#         "medical treatment",
#         "marijuana reclassification",
#         "Drug Enforcement Administration",
#         "Veterans Cannabis Project",
#         "cannabis companies"
#     ],
#     "headlines": [
#         "Major Implications Expected as Marijuana Restrictions May Ease",
#         "Health Agency Recommends Reclassification of Marijuana: A Step Towards National Legalization?",
#         "Marijuana Reclassification Could Boost Cannabis Businesses and Medical Treatment"
#     ],
#     "meta_titles": [
#         "Potential Easing of Marijuana Restrictions: What It Means",
#         "Reclassification of Marijuana: A Game Changer for Medical Treatment and Cannabis Businesses",
#         "Health Agency's Recommendation to Reclassify Marijuana: The Implications"
#     ],
#     "meta_descriptions": [
#         "The nation's top health agency recommends reclassifying marijuana, potentially paving the way for wider acceptance as a medical treatment and boosting cannabis businesses.",
#         "The possible reclassification of marijuana could have significant implications, including a potential path towards national legalization and increased scientific research into the drug's health benefits."
#     ]
# }


# {'headline_score': 'Yellow',
#  'headline_comment': 'The headline accurately describes the story, but it could be improved by including SEO keyword(s) for the story.',
#  'canonical_url_score': 'Green',
#  'canonical_url_comment': '',
#  'meta_title_score': 'Yellow',
#  'meta_title_comment': 'The meta title includes the SEO keyword(s) for the story, but it could be improved.',
#  'meta_description_score': 'Yellow',
#  'meta_description_comment': 'The meta description includes the SEO keyword(s) for the story, but it could be improved.'}



def convert_score_object_to_html(score_object, reCircLinks, story_object, ai_score_object):
  string_scores = ["Green", "Yellow", "Red"]
  ai_like_it_html = '<span class="Green">&#10004;</span><span style="margin-left:6px;">Looks good!</span>'

  # --------------------------------------------------------------------------------------------headline score
  # headline_score = score_object['headline']['value']
  # suggestion_text = "<div class=''><strong>Suggestions:</strong> " + score_object['headline']['suggestion'] + "</div>" if (score_object['headline']['suggestion'] != '' and score_object['headline']['score'] != 'Green') else ''
  suggestion_text = "<div class=''>" + score_object['headline']['suggestion'] + "</div>"

  # <span class="'+ score +'">&#10060;</span>
  # ---- ai scoring
  headline_ai_suggestions = "<div class=''><span class='"+ ai_score_object['headline_score'] +"' style='margin-right:6px;'>&#10008;</span><strong>Suggestion:</strong> "+ ai_score_object['headline_comment'] if (ai_score_object['headline_comment'] != '' and ai_score_object['headline_score'] != 'Green') else ai_like_it_html
  headline_html = """
    <li class="response_li li_{}">
      <h2>Headline</h2>
      <div class="">{}</div>
      {}
      {}
    </li>
  """.format(
      max([ score_object['headline']['score'], ai_score_object['headline_score']], key=lambda x: string_scores.index(x)),
      score_object['headline']['value'],
      suggestion_text,
      headline_ai_suggestions
  )

  # --------------------------------------------------------------------------------------------title score
  # suggestion_text = "<div class=''><strong>Suggestions:</strong> " + score_object['title']['suggestion'] + "</div>" if (score_object['title']['suggestion'] != '' and score_object['title']['score'] != 'Green') else ''
  suggestion_text = "<div class=''>" + score_object['title']['suggestion'] + "</div>"

  title_ai_alt_suggestions = "<h4 style='color: #a3a3a3;'>Alternate title suggestions:</h4><ul style='margin: 0px;font-size: 100%;margin-bottom:2px !important'>"
  for item in ai_score_object['meta_titles']:
      title_ai_alt_suggestions += '<li class="response_li" style="margin-bottom:2px !important">' + item + ' ('+ str(len(item)) +' chars)</li>'
  title_ai_alt_suggestions += "</ul>"


  # ---- ai scoring
  title_ai_suggestions = ''
  title_current_html = ''
  if score_object['title']['score_value'] > 0:
    title_ai_suggestions = "<div><span class='"+ ai_score_object['meta_title_score'] +"' style='margin-right:6px;'>&#10008;</span><strong>Suggestion:</strong> "+ ai_score_object['meta_title_comment']+"</div>" if (ai_score_object['meta_title_comment'] != '' and ai_score_object['meta_title_score'] != 'Green') else ai_like_it_html
    title_current_html = '<div><span class="">'+ score_object['title']['value'] +'</span></div>'

    # Suggestion: The meta title is missing. It should include primary keywords and accurately reflect the content of the story to improve SEO.


  title_html = """
    <li class="response_li li_{}">
      <h2>Meta Title</h2>
      {}
      {}
      {}
      {}
    </li>
  """.format(
      max([ score_object['title']['score'], ai_score_object['meta_title_score']], key=lambda x: string_scores.index(x)),
      title_current_html,
      suggestion_text,
      title_ai_suggestions,
      title_ai_alt_suggestions
  )

  # --------------------------------------------------------------------------------------------description score
  # suggestion_text = "<div class=''><strong>Suggestions:</strong> " + score_object['meta_description']['suggestion'] + "</div>" if (score_object['meta_description']['suggestion'] != '' and score_object['meta_description']['score'] != 'Green') else ai_like_it_html
  suggestion_text = "<div class=''>" + score_object['meta_description']['suggestion'] + "</div>"

  description_ai_alt_suggestions = "<h4 style='color: #a3a3a3;'>Alternate description suggestions:</h4><ul style='margin: 0px;font-size: 100%;margin-bottom:2px !important'>"
  for item in ai_score_object['meta_descriptions']:
      description_ai_alt_suggestions += '<li class="response_li" style="margin-bottom:2px !important">' + item + ' ('+ str(len(item)) +' chars)</li>'
  description_ai_alt_suggestions += "</ul>"

  description_ai_suggestions = ""
  if "meta_description_score" in ai_score_object:
    description_ai_suggestions = "<div class=''><span class='"+ ai_score_object['meta_description_score'] +"' style='margin-right:6px;'>&#10008;</span><strong>Suggestion:</strong> "+ ai_score_object['meta_description_comment'] if (ai_score_object['meta_description_comment'] != '' and ai_score_object['meta_description_score'] != 'Green') else ai_like_it_html

    description_html = """
      <li class="response_li li_{}">
        <h2>Meta Description</h2>
        <div class=""><strong>Current:</strong> {}</div>
        {}
        {}
        {}
      </li>
    """.format(
        max([ score_object['meta_description']['score'], ai_score_object['meta_description_score']], key=lambda x: string_scores.index(x)),
        score_object['meta_description']['value'],
        suggestion_text,
        description_ai_suggestions,
        description_ai_alt_suggestions
    )


  # --------------------------------------------------------------------------------------------canonical url score
  # ---- ai scoring
  canon_ai_suggestions = "<div class=''><span class='"+ ai_score_object['canonical_url_score'] +"' style='margin-right:6px;'>&#10008;</span><strong>Suggestion:</strong> "+  ai_score_object['canonical_url_comment'] if (ai_score_object['canonical_url_comment'] != '' and ai_score_object['canonical_url_score'] != 'Green') else ai_like_it_html

  canon_ai_html = """
    <li class="response_li li_{}">
      <h2>Custom Url</h2>
      <div class="">Current: {}</div>
      {}
    </li>
  """.format(
      ai_score_object['canonical_url_score'],
      get_story_canonical_url(story_object),
      canon_ai_suggestions
  )

  # --------------------------------------------------------------------------------------------reCirc score
  # print( score_object['recirc_links'] )
  # suggestion_text = "<div class=''><strong>Suggestions:</strong> " + score_object['recirc_links']['suggestion'] + "</div>" if (score_object['recirc_links']['suggestion'] != '') else ''
  suggestion_text = "<div class=''><strong>Suggestions:</strong> " + score_object['recirc_links']['suggestion'] + "</div>" if (score_object['recirc_links']['suggestion'] != '' and score_object['recirc_links']['score'] != 'Green') else ai_like_it_html
  recirc_links_html = ''

  recirc_suggest = ""
  if ( score_object['recirc_links']['need_to_add'] > 0):
    recirc_suggest = '<div><span class="{}">&#10008;</span><strong style="margin-left:6px;">Add:</strong> {} interstitials or story carousels</div>'.format(
        score_object['recirc_links']['score'],
        score_object['recirc_links']['need_to_add']
    )

  if len(reCircLinks) > 0:
    recirc_links_html = "<div style='margin-top:8px;'><strong>Suggested in-line links to include:</strong></div>"
    for recirc_link in reCircLinks:
      recirc_links_html += "<a href='https://www.washingtonpost.com"+ recirc_link['canonUrl'] +"' target='_blank'>"+ recirc_link['headline'] +"</a></BR>"

  recirc_html = """
    <li class="response_li li_{}">
      <h2>ReCirc Links</h2>
      {}
      <div class="">{}</div>
      {}
    </li>
  """.format(
      score_object['recirc_links']['score'],
      recirc_suggest,
      score_object['recirc_links']['value'],
      suggestion_text
  )

  # --------------------------------------------------------------------------------------------Internal Links score
  internal_links_html = "<div>ERROR</div>"
  try:
    suggestion_text = "<div class=''><strong>Suggestions:</strong> " + score_object['internal_links']['suggestion'] + "</div>" if (score_object['internal_links']['suggestion'] != '' and score_object['internal_links']['score'] != 'Green') else ai_like_it_html

    internal_links_suggest = ""
    if ( score_object['internal_links']['score'] != "Green" and score_object['internal_links']['value']['need_to_add'] > 0):
      internal_links_suggest = '<div><span class="{}">&#10008;</span><strong style="margin-left:6px;">Add:</strong> {} links</div>'.format(
          score_object['internal_links']['score'],
          score_object['internal_links']['value']['need_to_add']
      )

    # <div class=""><strong>Paragraphs:</strong> {}</div>
    # <div class=""><strong>Internal Links:</strong> {}</div>
    # <div class=""><strong>Paragraphs per Internal Link:</strong> {}</div>

    # print(json.dumps(score_object, indent=2))
    internal_links_html = """
      <li class="response_li li_{}">
        <h2>Internal Links</h2>
        {}
        {}
        {}
      </li>
    """.format(
        score_object['internal_links']['score'],
        internal_links_suggest,
        # score_object['internal_links']['value']['paragraph_count'],
        # score_object['internal_links']['value']['num_internal_links'],
        # score_object['internal_links']['value']['avg_paragraphs_per_internal_link'],
        suggestion_text,
        recirc_links_html
    )
  except KeyError as e:
    print("ERROR")
    print(e)
    internal_links_html = "***** Error: " #+ str(e)

  # --------------------------------------------------------------------------------------------Keywords

  ai_keywords_html = "" #"<ul>"
  for item in ai_score_object['keywords']:
      # ai_keywords_html += '<li style="margin-bottom:2px !important">' + item + '</li>'
      ai_keywords_html += '<div class="keyword_box">' + item + '</div>'

  keywords_text = """

    <li class="response_li" style="display: inline-block;"><h2>Keywords</h2>
    {}</li>
  """.format(
      ai_keywords_html,
      # non_ai_keywords_html
  )


  styles = """
  <style>
      .keyword_box {
          padding: 10px;
          width: max-content;
          min-width: 127px;
          float: left;
          text-align: center;
          margin-right: 10px;
          background-color: #e5e7eb;
          font-weight: bold;
          line-height: normal;
          margin-bottom: 10px;
      }
      .Green {
        color: #27ae60 !important;
      }
      .Yellow {
        color: #f1c40f !important;
      }
      .Red {
        color: #e74c3c !important;
      }
      .xtrot {
        float:left !important;
        margin-right:20px !important;

      }
      .response_ul {
        list-style: none !important;

      }
      .response_li {

        line-height:23px !important;
        margin-left: 20px;
        padding-left: 7px;
        list-style-type: circle;
      }
      .li_Green {
        list-style-type: "✔" !important;
        color: #27ae60 !important;
      }
      .li_Red {
        list-style-type: "✘" !important;
        color: #e74c3c !important;
      }
      .li_Yellow {
        list-style-type: "✘" !important;
        color: #f1c40f !important;
      }
      .container_div {
        font-size: 100%;
        margin-bottom: 50px;
      }
  </style>
  """

  html = """
  <div class="container_div">
    {}
    <ul class="response_ul">
    {}
    {}
    {}
    {}
    {}
    {}
    {}
    </ul>
  </div>
  """.format(
      styles,
      keywords_text,
      headline_html,
      title_html,
      description_html,
      canon_ai_html,
      internal_links_html,
      recirc_html,
  )

  return html

# NOT using this funciton anymore... (replaced by convert_score_object_to_html)
#    Keeping for reference
def render_results(score_object):
  """Renders the results of the analyze_seo_by_url function in HTML.
  Args: score_object: The score object returned by the analyze_seo_by_url function.
  Returns: An HTML string containing the results."""

  # print(score_object)

  headline_score = score_object['headline']['score']



  html = """
  <html>
  <head>
  <title>SEO Score</title>
  </head>
  <body>
  <h1>SEO Score</h1>
  <p>Your website's SEO score is XXXXX.</p>
  <p>The following are the individual scores for each category:</p>
  <ul>
  <li>Headline: ({} chars) {}</li>
  <li>Internal Links: ({} paragraphs per link) {}</li>
  <li>Recirc Links: ({}) {}</li>
  <li>Meta description: ({} chars) {}</li>

  </ul>
  </body>
  </html>
  """.format(
      score_object['headline']['value'],
      score_object['headline']['score'],
      score_object['internal_links']['value'],
      score_object['internal_links']['score'],
      score_object['recirc_links']['value'],
      score_object['recirc_links']['score'],
      score_object['meta_description']['value'],
      score_object['meta_description']['score'])
  return html


In [ ]:
# @title Story data functions (non-AI : keywords, etc.)

from textblob import TextBlob

TextBlob_model = KeyBERT()

def get_keywords_from_story_object(story_object):
  story_text = get_full_story_story_object(story_object)
  # print(story_text)
  # --------------test using KeyBERT
  # keybert_keywords = TextBlob_model.extract_keywords(story_text)

  # return TextBlob_model.extract_keywords(story_text, highlight=True)
  # keybert_keywords = TextBlob_model.extract_keywords(story_text, keyphrase_ngram_range=(1, 3), highlight=True)
  # print( "keybert:" + keybert_keywords)

  headline_text = get_story_headline( story_object )
  meta_title_text = get_story_title( story_object )
  description_text( story_object )

  fullstring = headline_text + '. ' + story_text
  keybert_keywords = get_keywords_from_string(fullstring)

  # blob = TextBlob(headline_text)
  # print(blob.noun_phrases)

  # textblob_nouns = [w for (w, pos) in TextBlob(headline_text).pos_tags if pos[0] == 'N']
  # print( "textblob_nouns:" + textblob_nouns)


  df = pd.DataFrame(keybert_keywords)
  df = df.loc[df[1] > 0.3]    # gets rid of keywords with score lower than .3
  df = remove_similar_words_df(df)

  keyword_array = []
  for index, row in df.iterrows():
    keyword_array.append(row[0])


  return keyword_array

  # for rating, keyword in r.get_ranked_phrases_with_scores():
  #   if rating > 5:
  #       print(rating, keyword)

  # return r.get_ranked_phrases_with_scores()
  # return r.get_word_degrees()
  # return r.get_word_frequency_distribution()
  # return r.extract_keywords_from_text(story_text)

# story_object = get_prism_data_from_url('https://www.washingtonpost.com/sports/2023/10/01/taylor-swift-kelce-chiefs-jets/')
# # get_keywords(story_object)
# story_html_string = get_keywords(story_object)

def get_keywords_from_string(mystring):
  return TextBlob_model.extract_keywords(mystring)

def get_noun_phrases_from_string(mystring):
  blob = TextBlob(mystring)
  return blob.noun_phrases

def get_word_counts_from_string(needle, haystack):
  return haystack.word_counts[needle]



import difflib

# def remove_similar_words_df(df):
#   # Get the first column of the dataframe.
#   first_column = df.iloc[:, 0]

#   # Create a list of words that are longer than 10 letters.
#   long_words = [word for word in first_column if len(word) > 10]

#   # Remove the rows that contain any of the long words.
#   df = df[~df.iloc[:, 0].isin(long_words)]

#   return df


# word_list = ['frog', 'toad', 'legalize', 'legalization', 'legalized']
# # word_list = ['marijuana', 'cannabis', 'weed']
# close_match = difflib.get_close_matches('legalize', word_list)
# # print( close_match)
# close_match.remove('legalize')

# result = []
# for i in word_list:
#     if i not in close_match:
#         result.append(i)
# # print(result)

# # >>> ['asf', 'bcd', 'cdf']


# def remove_similar_words_df(df):
#   xlist = df.values[:, 0]
#   new_array = []

from difflib import get_close_matches

# xlist = ['frog', 'toad', 'legalize', 'legalization', 'legalized', 'cat']
# xlist = keywords.values[:, 0].tolist()
# new_array = []

# while xlist:
#     word = xlist.pop()
#     new_array.append(word)
#     xlist = [x for x in xlist if x not in get_close_matches(word, xlist, cutoff=0.6)]

# print(new_array)

# df = keywords[~keywords[0].isin(new_array)]
# print(df)

def remove_similar_words_df(df):
  xlist = df.values[:, 0].tolist()
  new_array = []

  while len(xlist) > 0:
    word = xlist.pop()
    new_array.append(word)
    xlist = [x for x in xlist if x not in get_close_matches(word, xlist, cutoff=0.8)]

  # print( new_array)
  # if not new_array:
  #   return df

  # df = df[~df[0].isin(new_array)]
  new_df = df[df[0].isin(new_array)]
  return new_df



In [ ]:
# @title 3rd party API functions (non-AI: content_API, ReCirc tool, etc)

def get_content_from_content_elements(content_elements, headline = '', keep_html = False):
  full_story = [headline] if headline != '' else []
  for element in content_elements:
    if element['type'] == 'text':
      full_story.append(element['content'])

  soup = BeautifulSoup("".join(str(x) for x in full_story))
  full_text = soup.get_text() if keep_html != True else soup.html

  return full_text

# def get_full_story_from_search_results(search_object, add_headline = True):



def get_full_story_story_object(story_object, add_headline = True):
  # print( 'get_full_story_story_object' )
  # print( story_object )

  headline = get_story_headline( story_object )+'. ' if add_headline else ''
  return get_content_from_content_elements(get_content_elements(story_object), headline, True)

def get_full_story_html_api(story_object, add_headline = True ):
  headline = '<h1>' + get_story_headline( story_object ) + '</h1>' if add_headline else ''
  return get_content_from_content_elements(get_content_elements(story_object), headline)

def get_recirc_links_from_storyObject(story_object):
  headline = get_story_headline( story_object )
  section_url = get_story_section_path( story_object)
  canon_url = get_story_canonical_url( story_object)
  return get_recirc_links_from_ai(headline, section_url, canon_url)

def get_recirc_links_from_ai(headline, section, url):
  api_url = "https://{url}/api/wp-rec-articles/"
  url = url.replace('https://www.washingtonpost.com', '')
  payload = json.dumps({
    "input": {
      "text": headline,
      "topic": section,
      "url": url
      # "/elections/2023/09/20/trump-abortion-republicans-presidential/"
    }
  })

  # print( 'get_recirc_links_from_ai' )
  # print( payload )

  headers = {
    'Content-Type': 'application/json',
    'Authorization': 'Bearer ',
    'Cookie': 'connect.sid='
  }

  try:
    response = requests.request("POST", api_url, headers=headers, data=payload)
    response.raise_for_status()

    response_object = json.loads(response.json())

    # formatted_array = []
    # for story in response_object:

    # print(response.text)

    formatted_array = []
    for story in response_object:
      storyId = story['DocumentId']
      headline = story['DocumentTitle']['Text']
      pubDate = story['DocumentAttributes'][3]['Value']['StringValue']
      canonUrl = story['DocumentAttributes'][4]['Value']['StringValue']
      artType  = story['DocumentAttributes'][5]['Value']['StringValue']
      taxonomy = story['DocumentAttributes'][7]['Value']['StringListValue']

      answerText = story['DocumentExcerpt']['Text']
      excerpt = story['DocumentExcerpt']['Text']
      score = story['ScoreAttributes']['ScoreConfidence']

      # answerText = story['AdditionalAttributes']    # not all stories have this
      # print(headline)
      formatted_array.append( {
          "storyId": storyId ,
          "headline": headline,
          "pubDate": pubDate,
          "canonUrl": canonUrl,
          "artType": artType,
          "taxonomy": taxonomy,
          "answerText": answerText,
          "excerpt": excerpt,
          "score": score })

    return formatted_array

  except requests.exceptions.HTTPError as err:
    print(" ERROR LOADING RECIRC LINK ------")
    print( response )

    # raise SystemExit(err)

  # response = requests.request("POST", api_url, headers=headers, data=payload)
  # print( response )

  return []



In [ ]:
# @title PyTrend Functions!!!

# from: https://colab.research.google.com/github/FrontAnalyticsInc/data-winners/blob/main/analysis-trends/trends-and-forecasts.ipynb#scrollTo=c02a85f6

#install pytrends


#import the libraries
import pandas as pd
from pytrends.request import TrendReq
from datetime import datetime, timedelta

# use custom header
requests_args = {
    'headers': {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'
    }
}


# --- code 2
# this caused errors occasionaly
# pytrend = TrendReq(requests_args=requests_args)

pytrends = TrendReq(hl='en-US', tz=360, timeout=(10,25), retries=3, backoff_factor=0.3, requests_args={'verify':False})
pytrend = TrendReq()

#build model
# pytrend = TrendReq(requests_args=requests_args)

#provide your search terms
# kw_list=['"marijuana"']
# kw_list=['"marijuana"']
# pytrend.build_payload(kw_list=kw_list)
# pytrend.build_payload(kw_list=['pizza', 'bagel'], timeframe=['2022-09-04 2022-09-10', '2022-09-18 2022-09-24']))
# pytrend.get_historical_interest(kw_list, year_start=2018, month_start=1, day_start=1, hour_start=0, year_end=2018, month_end=2, day_end=1, hour_end=0, cat=0, geo='', gprop='', sleep=0)
# pytrend.build_payload(kw_list, cat=0, timeframe='today 5-y', geo='', gprop='')

# kw_list = ["Blockchain"]
# pytrend.build_payload(kw_list, cat=0, timeframe='today 5-y', geo='', gprop='')

#---------------------------get related queries (CJ NOT BAD)
# related_queries = pytrend.related_queries()
# related_queries.values()
# #build lists dataframes
# top = list(related_queries.values())[0]['top']


# rising = list(related_queries.values())[0]['rising']  #this has queries that will contain the word in it

#convert lists to dataframes
# dftop = pd.DataFrame(top)
# dfrising = pd.DataFrame(rising)

# dftop.head(50)


def gtrends_get_related_queries(query_array):
  # pytrend.build_payload(kw_list=['weed', 'marijuana', 'cannabis'], geo='US', gprop='', timeframe='now 1-H') today 1-m
  pytrend.build_payload(kw_list=query_array, geo='US', gprop='', timeframe='now 1-H')
  # pytrend.build_payload(kw_list=query_array, timeframe='today 5-y', geo='US', gprop='')

  related_queries = pytrend.related_queries()
  related_queries.values()
  top = list(related_queries.values())[0]['top']

  return pd.DataFrame(top)




def gtrends_get_suggestions(query_string):
  return pd.DataFrame(pytrend.suggestions(query_string))

# print( gtrends_get_related_queries(["marijuana restrictions"]) )
# print( gtrends_get_suggestions("marijuana restrictions") )

#get today's treniding topics
# trendingtoday = pytrend.today_searches(pn='US')

# Get Google Top Charts
# for querystring in trendingtoday:
#     q = querystring.split("q=")[1].split("&")[0].replace("+"," ")
#     print(q)

    #prints something like this
    # Pittsburgh Steelers
    # Pakistan vs Afghanistan
    # John Stamos
    # Liz Cheney



# --- code 3

def gtrends_get_interest_over_time(query_array, timeframe = 'today 1-m'):
  # 'now 1-H'
  pytrend.build_payload(kw_list=query_array, geo='US', gprop='', timeframe=timeframe)
  return pytrend.interest_over_time()





# pytrend.related_queries()
# pytrend.related_topics()
# pytrend.trending_searches(pn='united_states')
# pytrend.realtime_trending_searches(pn='US')
# pytrend.suggestions("marijuana")


# try:
#   print(x)
# except:
#   print("An exception occurred")



In [ ]:
# @title date functions

# keywords[:5]
# print( gtrends_get_related_queries(keywords[:5]) )
# gtrends_get_related_queries(['hurricane'])
# gtrends_get_related_queries(['Republican debate', 'Ron DeSantis', 'GOP presidential nomination', 'Donald Trump', 'Nikki Haley', 'Iowa caucuses'])
# print( type(keywords) )
# print( type(['marijuana', 'cannabis']) )

# pytrend.build_payload(kw_list=['hurricane'], geo='US', gprop='', timeframe='now 1-d')
# # pytrend.build_payload(kw_list=query_array, timeframe='today 5-y', geo='US', gprop='')

# related_queries = pytrend.related_queries()
# related_queries.values()
# print(related_queries)
# top = list(related_queries.values())[0]['top']
# pd.DataFrame(top)
import datetime
from datetime import date
from calendar import monthrange
# pytrend.build_payload(kw_list=['hurricane', 'Winter storm'], geo='US', gprop='', timeframe='now 1-d')
# pytrend.multirange_interest_over_time()
def get_last_date_of_month(year: int, month: int) -> date:
    """Given a year and a month returns an instance of the date class
    containing the last day of the corresponding month.

    Source: https://stackoverflow.com/questions/42950/get-last-day-of-the-month-in-python
    """
    return date(year, month, monthrange(year, month)[1])

def convert_dates_to_timeframe(start: date, stop: date) -> str:
    """Given two dates, returns a stringified version of the interval between
    the two dates which is used to retrieve data for a specific time frame
    from Google Trends.
    """
    return f"{start.strftime('%Y-%m-%d')} {stop.strftime('%Y-%m-%d')}"

# start_date = date(2023, 12, 1)
# stop_date = get_last_date_of_month(2023, 12)

# print(convert_dates_to_timeframe(start_date, stop_date) )


In [ ]:
# @title AI QUERY : ai_score_story() FUNCTION
# string
# number
# boolean
# null/empty
# object
# array

def ai_score_story(story_object, story_html_string):
  global open_ai_model, open_ai_max_tokens, open_ai_temp
  seo_custom_functions = [
      {
          'name': 'analyze_story_seo',
          'description': 'Analyze and score the SEO quality of parts of the story from the input text',
          'parameters': {
              'type': 'object',
              'properties': {
                  'keywords': {
                      'type': 'string',
                      'description': 'List of top 10 SEO keywords from story, separated by commas, in order of importance.'
                  },
                  'meta_titles': {
                      'type': 'string',
                      'description': 'Write 3 different meta titles for the story using the top SEO keywords when able, separated by **.'
                  },
                  'meta_description_score': {
                      'type': 'string',
                      "enum": ["Green", "Yellow", "Red"],
                      'description': 'Does the meta description accurately describe the story?'
                  },
                  'meta_description_comment': {
                      'type': 'string',
                      'description': 'Add any suggestions to improve meta description.'
                  },
                  'meta_description_examples': {
                      'type': 'string',
                      'description': 'Write 3 different meta descriptions, each around 150 characters in length, using many top SEO keywords, separated by **.'
                  }
              },
              "required": [
                  "keywords",
                  "meta_titles",
                  "meta_description_score",
                  "meta_description_comment",
                  "meta_description_examples"
              ]
          }
      }
  ]
  # print("Meta title: ")
  # print(get_story_title( story_object ))
  prompt = """
  Headline: {}
  Story:
  {}
  Meta Title: {}
  Meta Description: {}

  Answer the following:
  1. What are the TOP 10 SEO keywords for the story?
  2. Write 3 meta titles, separated by **. Each should be:
    - 50-60 characters in length. IMPORTANT!
    - Use top SEO keywords as early in meta title as possible
  3. SCORE the meta description according to scale of "Red" = Very good, "Yellow" = Needs work
      and "Red" = Needs improvement.
  4. Offer a suggestion on how to improve meta description.
  5. Write 3 meta descriptions, around 150 characters in length, separated by **. Each should be:
    - Different from the current description, headline or title
    - Use as many top SEO keywords as possible
    - Be clear and concise: What’s the story about?

  IMPORTANT: Make the descriptions each around 150 characters in length.

  At the end of the conversation, respond with "<|DONE|>".""".format(
      get_story_headline(story_object),
      story_html_string,
      get_story_title( story_object ),
      get_story_description( story_object )
  )

  try:
    response = openai.chat.completions.create(
        # model="gpt-3.5-turbo",
        # model="gpt-4",
        # model="gpt-3.5-turbo-16k",
        model="gpt-4-1106-preview",
        # model=open_ai_model,
        messages=[
            {"role": "system", "content": "You are a helpful but meticulous news copy editor, writer and SEO expert."},
            {"role": "user", "content": prompt}
        ],
        functions = seo_custom_functions,
        function_call = 'auto',
        temperature=open_ai_temp,
        # max_tokens=open_ai_max_tokens,
    )

    print('------------AI response')
    print(response)
    response_json = json.loads(response.choices[0].message.function_call.arguments)
    print(response_json)
    response_keywords = response_json['keywords'].split(', ')
    response_meta_descriptions = response_json['meta_description_examples'].split('**')
    response_meta_titles = response_json['meta_titles'].split('**')
    print("meta 1")
    response_json['keywords'] = [s.strip() for s in response_keywords]
    response_json['meta_descriptions'] = [s.strip() for s in response_meta_descriptions][:3]
    response_json['meta_titles'] = [s.strip() for s in response_meta_titles][:3]
    print("meta 2")
    return response_json

  except openai.APIError as e:
    #Handle API error here, e.g. retry or log
    print(f"----ERROR: OpenAI API returned an API Error: {e}")
    # return json of error
    return { 'error': e }
    pass

  return {}






In [ ]:
# @title quicktests -

import IPython

# url = "https://www.washingtonpost.com/health/2023/12/21/colon-cancer-increasing-young-adults/"
url = "https://www.washingtonpost.com/politics/2023/12/27/michigan-supreme-court-trump-2024-primary-ballot/"

# story_object = get_prism_data_from_url(url)
# story_object

# get_story_title(story_object)
# get_story_canonical_url(story_object)

# get_prism_search_data_by_query('taylor swift')

# score_html = get_html_score_from_url( url )
# IPython.display.HTML(score_html)

# interface_search_stories_by_query("Lewiston")

# _analyze_title(story_object)

# score_object, this_story_object = analyze_seo_by_story_object(story_object)
# score_object

# get_recirc_links_from_storyObject(story_object)


'michigan-supreme-court-trump-2024-primary-ballot'

# Interface

In [ ]:
# @title MAIN WORKING INTERFACE (start this one)

import gradio as gr

def int_selectRowResult( theIndex):
  print('--------int_selectRowResult')
  print(theIndex)

def on_select(evt: gr.SelectData):
  canonUrl = get_story_canonical_url( search_results_array[evt.index[0]] )
  return canonUrl, gr.Tabs.update(selected=0), evt.index[0]

def analyze_selected_object(theRow):
  print("ROW: "+ theRow)
  global search_results_array
  return get_html_score_from_story_object( search_results_array[int(theRow)] )

with gr.Blocks() as demo:
  gr.Markdown("Analyze <B>SEO</B> score for Wapo story.")
  with gr.Tabs() as tabs:
    with gr.TabItem("URL", id=0):
      with gr.Row():
        text_url_input = gr.Textbox(label="Enter URL",
            max_lines=1,
            placeholder="https://www.washingtonpost.com/dc-md-va/2023/10/16/census-race-eliminate-race-box/",
            container=False,
        )
        analyze_button = gr.Button("Analyze URL")

    with gr.TabItem("Search story", id=1):
        with gr.Row():
          story_search_input = gr.Textbox(label="Enter Search",
              max_lines=1,
              placeholder="What do you want to search for? (one word only please)",
              container=False,
          )
          story_search_button = gr.Button("Search")

        with gr.Row():
          search_results_output2 = gr.Dataframe(label="Search Results",
              interactive=False,
              height=500,
          )
          selectedRow = gr.Textbox(
            visible=False
          )

  with gr.Tabs() as xtabs:
    with gr.TabItem("Analysis", id=3):
      with gr.Row():
        text_output = gr.HTML()
    with gr.TabItem("Keywords", id=4, interactive=False):
      with gr.Row():
        keyword_output = gr.HTML()
        process_keywords_button = gr.Button("Keyword Trends")

  search_results_output2.select(on_select, None, [text_url_input, tabs, selectedRow], queue=False).then(
      analyze_selected_object, selectedRow, text_output
  )

  analyze_button.click(get_html_score_from_url, inputs=text_url_input, outputs=text_output, queue=False)
  story_search_button.click(interface_search_stories_by_query, inputs=story_search_input, outputs=search_results_output2, queue=False)

# demo.queue().launch(debug=True)
demo.queue().launch()

# Testing

In [ ]:
# @title Testing out google speed

# https://huggingface.co/spaces/upthrustinc/seoAnalyzerGPT
# https://huggingface.co/spaces/upthrustinc/seoAnalyzerGPT/tree/main

# CJ THIS WORKS PRETTY GOOD... TELLS BASIC SEO STUFF THAT IS NOT CONTENT RELATED (SO DO WE NEED IT HERE?)
#     putting on back shelf for now as it does not fit in scope of project
#      also note that the csv file helps give more info on each issue... its pretty neat.
from google.colab import userdata

def generate_response(website_url, url = "https://www.googleapis.com/pagespeedonline/v5/runPagespeed"):
    print("Website: " + website_url)
    print()
    name = website_url.split("//")[1].split(".")[1] # Get the name of the website

    params = {
        "url": website_url,
        "key": userdata.get('GOOGLE_API_KEY_2'),
        "category": ["performance", "accessibility", "best_practices", "seo"]
    }

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()

        data = response.json()

        # Process the data as needed
        return data

    except requests.exceptions.RequestException as e:
        print("Error:", e)

def process_data(data):
    audits = [data["lighthouseResult"]["audits"][i] for i in data["lighthouseResult"]["audits"]]
    audits_names = [i["title"] for i in audits]

    scoresdisplays = [data["lighthouseResult"]["audits"][i]["scoreDisplayMode"] for i in data["lighthouseResult"]["audits"]]

    issues = []
    for i in audits:
        if i["scoreDisplayMode"] != "notApplicable" and (i["score"] != 1 and i["score"] != None) and "details" in i.keys() and i["scoreDisplayMode"] != "informative":
            title = i["title"]
            desc = i["description"]
            item = i["details"]["items"][0]
            typeOfIssue = i["details"]["type"]
            dicto = {"title": title, "description": desc, "item": item, "type": typeOfIssue}
            issues.append(dicto)
            print(title)
            print(i["details"]["type"])
            question = f"Title: {title}\nDescription: {desc}\nItem: {item}"
            #print(answer_question(df, question=question, debug=False))
            print("***********************************")
    return issues

googleSpeedInsight = generate_response("https://www.washingtonpost.com/health/2023/08/31/marijuana-reclassification-schedule-iii-hhs-dea/")
googleSpeedInsight_Issues = process_data(googleSpeedInsight)
print( googleSpeedInsight )


# googleSpeedInsight

In [ ]:
# @title TESTING recirc

# fullpath = "https://washpost.arcpublishing.com/ellipsis/api/v1/tags/{key}/automated"
# fullpath = "https://api.taxonomy.platform.aws.wapo.pub/api/v1/article?url=/sports/2023/10/01/taylor-swift-kelce-chiefs-jets/"
# response = requests.get(fullpath)
# print(response.content)


# search_results = get_prism_search_data_by_query("Lewiston")
# search_object



# cleaned_results.append({
#         "name": "Carol",
#         "age": 22
#     })
  #     "text": "Taylor Swift comes to Chiefs-Jets, and the NFL gets an even bigger spotlight",
  #     "topic": "/sports",
  #     "url": "https://www.washingtonpost.com/sports/2023/10/01/taylor-swift-kelce-chiefs-jets/"

# my_json_object = json.loads(response.content)
# https://api.taxonomy.platform.aws.wapo.pub/api/v1/article?url=/sports/2023/10/01/taylor-swift-kelce-chiefs-jets/

# response_object = get_recirc_links_from_ai("Taylor Swift comes to Chiefs-Jets, and the NFL gets an even bigger spotlight", "/sports", "https://www.washingtonpost.com/sports/2023/10/01/taylor-swift-kelce-chiefs-jets/")
# response_object[0]









In [ ]:
# @title test AI summarizing a story

# search_results['content_elements'][0]
story_html_string = get_full_story_html_api(search_results['content_elements'][6])
story_html_string

prompt = """
STORY:
{}

Please summarize the story in a concise bulleted list, including any specific facts and details.

".""".format(story_html_string)


# story_html_string
response = openai.completions.create(
    model="gpt-3.5-turbo",
    # model="gpt-4",
    messages=[
        {"role": "system", "content": "You are an expert at summarizing and pulling important data from stories."},
        {"role": "user", "content": prompt}
    ],
    temperature=0.0,
)

print(f"summary: {response.choices[0].message}\n")



In [ ]:
# non_ai_keywords = get_keywords_from_story_object(story_object)
# non_ai_keywords


['marijuana', 'cannabis', 'legalizing', 'legalization', 'reclassification']

In [ ]:
# score_object = analyze_seo_by_url("https://www.washingtonpost.com/health/2023/08/31/marijuana-reclassification-schedule-iii-hhs-dea/")
score_object['recirc_links']
# story_object

{'score': 'Red',
 'value': 'You currently have 0 interstitials and 0 story carousels for your 31 paragraph story.',
 'suggestion': 'Ideally have at least 2 interstitials or story carousels for articles at least 12 paragraphs long.'}

In [ ]:
# @title New HTML Results TEST

import IPython

# # url = "https://www.washingtonpost.com/nation/2023/10/31/trump-family-new-york-civil-trial/"
url = "https://www.washingtonpost.com/politics/2023/12/27/michigan-supreme-court-trump-2024-primary-ballot/"
score_html = get_html_score_from_url( url )
IPython.display.HTML(score_html)

In [ ]:
# @title pretty print conversation (NOT SURE I NEED)

from termcolor import colored

def pretty_print_conversation(messages):
    role_to_color = {
        "system": "red",
        "user": "green",
        "assistant": "blue",
        "function": "magenta",
    }

    for message in messages:
        if message["role"] == "system":
            print(colored(f"system: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "user":
            print(colored(f"user: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "assistant" and message.get("function_call"):
            print(colored(f"assistant: {message['function_call']}\n", role_to_color[message["role"]]))
        elif message["role"] == "assistant" and not message.get("function_call"):
            print(colored(f"assistant: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "function":
            print(colored(f"function ({message['name']}): {message['content']}\n", role_to_color[message["role"]]))

In [ ]:
story_object = get_prism_data_from_url("https://www.washingtonpost.com/health/2023/08/31/marijuana-reclassification-schedule-iii-hhs-dea/")
# get_keywords(story_object)
story_html_string = get_full_story_story_object(story_object)
story_html_string

In [ ]:
# story_object = get_prism_data_from_url("https://www.washingtonpost.com/health/2023/08/31/marijuana-reclassification-schedule-iii-hhs-dea/")
# scoreObject = analyze_seo_by_url("https://www.washingtonpost.com/health/2023/08/31/marijuana-reclassification-schedule-iii-hhs-dea/")
pd.DataFrame(get_keywords_from_story_object(story_object))
